In [ ]:
pip install dash

In [2]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import numpy as np
import joblib
import json

In [3]:
# Load required files
ohe_dict = pd.read_pickle('encoders.pkl')
bins = json.load(open('bins.json'))
model = pd.read_pickle('xgboost_all_cities.pkl')

In [4]:

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Define the layout
app.layout = html.Div(
    style={'display': 'grid', 'gridTemplateColumns': '1fr 1fr', 'gap': '20px', 'padding': '20px'},
    children=[
        html.H1("Apartment Price Prediction", style={'gridColumn': 'span 2', 'textAlign': 'center'}),

        html.Label("Type of building"),
        dcc.Dropdown(
            id='type',
            options=[
              {'label': 'Block of Flats', 'value': 'blockOfFlats'},
              {'label': 'Apartment Building', 'value': 'apartmentBuilding'},
              {'label': 'Tenement', 'value': 'tenement'}
            ],
            value='blockOfFlats'
        ),

        html.Label("City"),
        dcc.Dropdown(
            id='city',
            options=[
                {'label': city.capitalize(), 'value': city} for city in [
                    'bialystok', 'bydgoszcz', 'czestochowa', 'gdansk', 'gdynia', 'katowice',
                    'krakow', 'lodz', 'lublin', 'poznan', 'radom', 'rzeszow', 'szczecin',
                    'warszawa', 'wroclaw'
                ]
            ],
            value='warszawa'
        ),

        html.Label("Size of Apartment in Square Meters"),
        dcc.Input(id='squareMeters', type='number', min=10, max=200, value=50),

        html.Label("Floor"),
        dcc.Input(id='floor', type='number', min=1, max=20, value=5),

        html.Label("Building Build Year"),
        dcc.Input(id='buildYear', type='number', min=1850, max=2025, value=1980),

        html.Label("Distance to City Center (km)"),
        dcc.Input(id='centreDistance', type='number', min=0, max=20, step=0.01, value=5),

        html.Label("Number of Points of Interest in 500m range from the apartment"),
        dcc.Input(id='poiCount', type='number', min=0, max=200, value=50),

        html.Label("Clinic Distance (km)"),
        dcc.Input(id='clinicDistance', type='number', min=0, max=10, step=0.001, value=0.500),

        html.Label("Kindergarten Distance (km)"),
        dcc.Input(id='kindergartenDistance', type='number', min=0, max=10, step=0.001, value=0.500),

        html.Label("College Distance (km)"),
        dcc.Input(id='collegeDistance', type='number', min=0, max=10, step=0.001, value=0.500),

        html.Label("Pharmacy Distance (km)"),
        dcc.Input(id='pharmacyDistance', type='number', min=0, max=10, step=0.001, value=0.500),

        html.Label("Property Condition"),
        dcc.Dropdown(
            id='condition',
            options=[
                {'label': 'Standard', 'value': 'standard'},
                {'label': 'Premium', 'value': 'premium'},
                {'label': 'Low', 'value': 'low'}
            ],
            value='standard'
        ),

        html.Label("Has Parking Space?"),
        dcc.RadioItems(id='hasParkingSpace', options=[{'label': 'Yes', 'value': 'yes'}, {'label': 'No', 'value': 'no'}], value='yes'),

        html.Label("Has Balcony?"),
        dcc.RadioItems(id='hasBalcony', options=[{'label': 'Yes', 'value': 'yes'}, {'label': 'No', 'value': 'no'}], value='yes'),

        html.Label("Has Elevator?"),
        dcc.RadioItems(id='hasElevator', options=[{'label': 'Yes', 'value': 'yes'}, {'label': 'No', 'value': 'no'}], value='yes'),

        html.Button('Predict', id='predict-button', n_clicks=0, style={'gridColumn': 'span 2', 'marginTop': '20px'}),

        html.Div(id='prediction-output', style={'gridColumn': 'span 2', 'textAlign': 'center', 'fontSize': '20px', 'fontWeight': 'bold'})
    ]
)



In [5]:
# Prediction function
def predict_price(input_data):
    df = pd.DataFrame([input_data])

    # Bin distance columns
    columns = ['clinicDistance', 'kindergartenDistance', 'collegeDistance', 'pharmacyDistance']
    for col in columns:
        df[f'{col}_binned'] = pd.cut(df[col], bins=bins['distance_bins'], labels=False, include_lowest=True)
    df = df.drop(columns=columns)

    df['centreDistance_binned'] = pd.cut(df['centreDistance'], bins=bins['centre_distance_bins'], labels=False, include_lowest=True)
    df = df.drop(columns='centreDistance')

    # Encode city
    city_encoder = ohe_dict['city_encoder']
    city_encoded = city_encoder.transform(df[['city']])
    encoded_city_df = pd.DataFrame(city_encoded, columns=city_encoder.get_feature_names_out(['city']))
    df = pd.concat([df.drop(columns='city'), encoded_city_df], axis=1)

    # Encode categorical columns
    cat_encoder = ohe_dict['cat_column_encoder']
    cat_columns = df.select_dtypes('object').columns
    transformed = cat_encoder.transform(df[cat_columns])
    encoded_df = pd.DataFrame(transformed, columns=cat_encoder.get_feature_names_out(cat_columns))
    df = pd.concat([df.drop(columns=cat_columns), encoded_df], axis=1)

    # Predict price
    price = model.predict(df)
    price_per_m2 = np.exp(price)

    return f"Predicted Price per m²: {price_per_m2[0]:.2f} PLN"

In [6]:
# Dash callback to make predictions
@app.callback(
    Output('prediction-output', 'children'),
    Input('predict-button', 'n_clicks'),
    State('type', 'value'),
    State('city', 'value'),
    State('squareMeters', 'value'),
    State('floor', 'value'),
    State('buildYear', 'value'),
    State('centreDistance', 'value'),
    State('poiCount', 'value'),
    State('clinicDistance', 'value'),
    State('kindergartenDistance', 'value'),
    State('collegeDistance', 'value'),
    State('pharmacyDistance', 'value'),
    State('condition', 'value'),
    State('hasParkingSpace', 'value'),
    State('hasBalcony', 'value'),
    State('hasElevator', 'value'),
)
def update_prediction(n_clicks, type, city, squareMeters, floor, buildYear, centreDistance, poiCount,
                      clinicDistance, kindergartenDistance, collegeDistance, pharmacyDistance,
                      condition, hasParkingSpace, hasBalcony, hasElevator):
    if n_clicks == 0:
        return ""

    input_data = {
        "type": type,
        "city": city,
        "squareMeters": squareMeters,
        "floor": floor,
        "buildYear": buildYear,
        "centreDistance": centreDistance,
        "poiCount": poiCount,
        "clinicDistance": clinicDistance,
        "kindergartenDistance": kindergartenDistance,
        "collegeDistance": collegeDistance,
        "pharmacyDistance": pharmacyDistance,
        "condition": condition,
        "hasParkingSpace": hasParkingSpace,
        "hasBalcony": hasBalcony,
        "hasElevator": hasElevator
    }

    return predict_price(input_data)

In [7]:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>